# Domain –Retail

focus –Optimize Book RENT


Business challenge/requirement


BookRent is the largest online and offline book rental chain in India. 

TheCompany charges a fixed fee per month plus rental per book. 
So,the company makes more money when user rent more books. 

You as an ML expert have to model recommendation engine so that user gets recommendation of books based on the behaviorof similar users. 

This will ensure that users are renting books based on their individual taste.
Company is still unprofitable and is looking to improve both revenue and profit.

# Data volume-Approx 


1 M  records –file BX-Book-Ratings.csv and 2 more. But only 10K records will be usedFields in Data

•user_id: Unique Id of the User
    
•isbn: International Standard Book Number is a unique numeric commercial book identifier
    
•rating: rating given by userAdditional information-NA


In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [8]:
#Reading users file:
#u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('BX-Users.csv',encoding='latin-1')

D:\Users\Plaban_Nayak\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
users.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
#Reading ratings file:
#r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('BX-Book-Ratings.csv',encoding='latin-1')
ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [12]:
#Reading items file:
items = pd.read_csv('BX-Books.csv',encoding='latin-1')
items.head()

D:\Users\Plaban_Nayak\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


#### Users

In [18]:
users.isnull().sum()

user_id          0
Location         1
Age         110763
dtype: int64

In [20]:
users.Age.fillna('0',inplace=True)

In [21]:
print(users.shape)

users.head()

(278859, 3)


,user_id,Location,Age
0,1,"nyc, new york, usa",0
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",0
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",0


#### Ratings

In [22]:
print(ratings.shape)
ratings.head()

(1048575, 3)


,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


#### Items

In [24]:
print(items.shape)
items.head()

(271379, 5)


,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [26]:
from sklearn.model_selection import train_test_split
ratings_train,ratings_test = train_test_split(ratings,test_size=0.25,random_state=1)

In [27]:
ratings_train.shape, ratings_test.shape

((786431, 3), (262144, 3))

#### Building collaborative filtering model from scratch

We will recommend movies based on user-user similarity and item-item similarity. For that, first we need to calculate the number of unique users and movies.

In [29]:
n_users = ratings.user_id.nunique()
n_items = ratings.isbn.nunique()

print(n_users)
print(n_items)

95513
322102


In [31]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.isbn.unique().shape[0]

print(n_users)
print(n_items)

95513
322102


#### create a user-item matrix which can be used to calculate the similarity between users and items.

In [30]:
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

MemoryError: 